# Executing Circuit

In this section, we will explain how to execute quantum circuits on the optical quantum computer of MQC3.

## Executing circuit representation

First, we create the circuit below.

![circuit](_images/circuit_repr_sample_circuit.svg)

In [ ]:
from math import pi

from mqc3.circuit import CircuitRepr
from mqc3.circuit.ops.intrinsic import ControlledZ, Displacement, Measurement, PhaseRotation

circuit = CircuitRepr("circuit")
circuit.Q(0) | PhaseRotation(phi=pi / 4) | Displacement(1, -1)
circuit.Q(0, 1) | ControlledZ(g=1)
circuit.Q(0) | Measurement(theta=0)
circuit.Q(1) | Measurement(theta=pi / 2)

circuit

### Configuring client

To execute a circuit from MQC3 SDK, you must create a {py:class}`~.mqc3.client.MQC3Client`, which specifies the connection point, API token, and execution parameters.
Let's create {py:class}`~.mqc3.client.MQC3Client` as follows.

In [ ]:
from mqc3.client import MQC3Client

client = MQC3Client()

You must set your API token in the `token` attribute.

In [ ]:
# client.token = "YOUR_API_TOKEN"

You can execute a circuit on either the optical quantum computer or an emulator by setting the `backend` attribute.
When the `backend` is set to`"qpu"`, the circuit is executed on the optical quantum computer,
and when the `backend` is set to `"emulator"`, it is executed on the emulator.

In [ ]:
# client.backend = "qpu"
client.backend = "emulator"

You can specify how many times the circuit will be executed by setting the `n_shots` attribute.

In [ ]:
# Run the circuit for hundred times
client.n_shots = 100

### Executing circuit 

You can execute the circuit with {py:func}`~mqc3.execute.execute`.
This function takes a circuit and a client as arguments and returns the execution result.

In [ ]:
from mqc3.execute import execute

circuit_result = execute(circuit, client)

In [ ]:
type(circuit_result)

{py:class}`~mqc3.execute.ExecutionResult` has the following attributes.

| Attribute | Type | Description |
| --------- | ---- | ----------- |
| {py:attr}`~mqc3.execute.ExecutionResult.total_time`         | `timedelta` | The total time to execute a representation. |
| {py:attr}`~mqc3.execute.ExecutionResult.execution_time`     | `timedelta` |The time to execute a representation. |
| {py:attr}`~mqc3.execute.ExecutionResult.input_repr`         | {py:class}`~mqc3.circuit.CircuitRepr`, {py:class}`~mqc3.graph.GraphRepr` or {py:class}`~mqc3.machinery.MachineryRepr` | Executed representation.  |
| {py:attr}`~mqc3.execute.ExecutionResult.execution_result`   | {py:class}`~mqc3.circuit.CircuitResult`, {py:class}`~mqc3.graph.GraphResult` or {py:class}`~mqc3.machinery.MachineryResult` |Execution result. |
| {py:attr}`~mqc3.execute.ExecutionResult.client_result`      | {py:class}`~mqc3.client.abstract.AbstractClientResult` | Inherent result property for each client. |
| {py:attr}`~mqc3.execute.ExecutionResult.n_shots`            | `int` | The number of shots. |

In [ ]:
from pprint import pprint

pprint(circuit_result)

When you use a {py:class}`~.mqc3.client.MQC3Client`, {py:attr}`~mqc3.execute.ExecutionResult.client_result` is {py:class}`~mqc3.client.MQC3ClientResult`.

In [ ]:
type(circuit_result.client_result)

{py:class}`~mqc3.client.MQC3ClientResult` has the following attributes.

| Attribute | Type | Description |
| --------- | ---- | ----------- |
| {py:attr}`~mqc3.client.MQC3ClientResult.execution_details` | {py:class}`~mqc3.client.mqc3_client.ExecutionDetails` | Versions and timestamps related to a job execution. |
| {py:attr}`~mqc3.client.MQC3ClientResult.circuit_result`    | {py:class}`~mqc3.circuit.CircuitResult` or None | Measurement results of a circuit representation. |
| {py:attr}`~mqc3.client.MQC3ClientResult.graph_result`      | {py:class}`~mqc3.graph.GraphResult` or None | Measurement results of a compiled graph representation. |
| {py:attr}`~mqc3.client.MQC3ClientResult.machinery_result`  | {py:class}`~mqc3.machinery.MachineryResult` | Measurement results of a machinery representation. |
| {py:attr}`~mqc3.client.MQC3ClientResult.compiled_graph`    | {py:class}`~mqc3.graph.GraphRepr` or None | A compiled graph representation. If {py:class}`~mqc3.graph.GraphRepr` is input, this represents the graph with `n_local_macronodes` set to 101 for machinery execution. If {py:class}`~mqc3.circuit.CircuitRepr` is input, this is the compiled graph of the {py:class}`~mqc3.graph.GraphRepr` converted from the circuit. |
| {py:attr}`~mqc3.client.MQC3ClientResult.compiled_machinery` | {py:class}`~mqc3.machinery.MachineryRepr` | A compiled machinery representation. If  {py:class}`~mqc3.machinery.MachineryRepr` is input, this is the input itself. Otherwise, this is the result of converting the {py:attr}`~mqc3.client.MQC3ClientResult.compiled_graph` into a {py:class}`~mqc3.machinery.MachineryRepr`. |
| {py:attr}`~mqc3.client.MQC3ClientResult.execution_result` | {py:class}`~mqc3.circuit.CircuitResult`, {py:class}`~mqc3.graph.GraphResult` or {py:class}`~mqc3.machinery.MachineryResult` | Execution result. |
| {py:attr}`~mqc3.client.MQC3ClientResult.wait_time` | `timedelta` or None | The waiting time until the job starts to run. |
| {py:attr}`~mqc3.client.MQC3ClientResult.compile_time` | `timedelta` or None | The time that a representation is compiled. |
| {py:attr}`~mqc3.client.MQC3ClientResult.execution_time`  | `timedelta` or None | The time to execute a representation. |
| {py:attr}`~mqc3.client.MQC3ClientResult.total_time`  | `timedelta` or None | The total time to execute a representation. |
| {py:attr}`~mqc3.client.MQC3ClientResult.n_shots`   | `int` | The number of shots. |

When you execute the circuit representation, you can get the result of the circuit representation for any shot by using index access.

In [ ]:
pprint(circuit_result[0])  # Get the 0-th shot result

In [ ]:
pprint(circuit_result[1])  # Get the 1-st shot result

The `index` parameter of each {py:class}`~mqc3.circuit.result.CircuitOperationMeasuredValue` object represents the mode index.

(sec:execute-graph-repr)=

## Executing graph representation

To show an example of executing graph representations, we will use the `compiled_graph` attribute of `circuit_result`. {py:func}`~mqc3.execute.execute` can be used with the graph representation in the same way as with the circuit representation.

```{note}
`compiled_graph` is used to simplify the example by omitting the process of constructing the graph representation. In practice, users typically construct graph representations themselves.

In the present example, since the circuit representation was executed before, the result of the graph representation, which has been converted to the circuit representation result, is already available as the `graph_result` attribute. Therefore, executing the following code is, in fact, unnecessary.
```

In [ ]:
from mqc3.client import MQC3ClientResult

assert isinstance(circuit_result.client_result, MQC3ClientResult)
assert circuit_result.client_result.compiled_graph is not None
graph_result = execute(circuit_result.client_result.compiled_graph, client)

pprint(graph_result)

Since you execute the graph representation, `circuit_result` is ``None``.

When you execute the graph representation, you can get the result of the graph representation for any shot by using index access.

In [ ]:
print(graph_result[0])  # Get the 0-th shot result

## Executing machinery representation

To show an example of executing a machinery representation, we will use the `compiled_machinery` attribute of `circuit_result`. {py:func}`~mqc3.execute.execute` can be used with the machinery representation in the same way as with the circuit representation.

```{note}
When executing a machinery representation, `n_local_macronodes` must be set to 101; otherwise, an error is raised.
```

```{note}
Similarly to what was noted in {ref}`sec:execute-graph-repr`, the following code can also be replaced by accessing `circuit_result.machinery_result`.
```

In [ ]:
assert isinstance(circuit_result.client_result, MQC3ClientResult)
assert circuit_result.client_result.compiled_machinery is not None
machinery_result = execute(circuit_result.client_result.compiled_machinery, client)

pprint(machinery_result)

Since you execute the machinery representation, three attributes (`circuit_result`, `compiled_graph`, and `graph_result`) are ``None``.

When you execute the machinery representation, you can get the result of the machinery representation for any shot by using index access.

In [ ]:
print(machinery_result[0])  # Get the 0-th shot result

(sec:execution-resource-and-initialized-states)=

## Resource and initialized states

The current MQC3 only utilizes squeezed states as resources, as described in {ref}`sec:machinery`.
These are used for the mode initialization, as explained in {ref}`sec:measurement-and-initialization`, and the initialized modes are also squeezed states.
The following table presents the parameters of the resource modes and initialized modes for two backends, qpu and emulator.

<style>
.scroll-table {
    overflow-x: auto;
}
table {
    border-collapse: collapse;
}
td, th {
    border: 1px solid gray;
    padding: 8px;
}
</style>

<div class="scroll-table">
<table>
<tr>
<td rowspan="2" align="center">Representation</td>
<td rowspan="2" align="center">Parameter</td>
<td colspan="2" align="center">qpu</td>
<td colspan="2" align="center">emulator</td>
</tr>
<tr>
<td align="center">Resource</td>
<td align="center">Initialized</td>
<td align="center">Resource</td>
<td align="center">Initialized</td>
</tr>
<tr>
<td rowspan="3" align="center">Circuit</td>
<td align="center">Squeezing level</td>
<td rowspan="3" align="center">(there is no concept of resource)</td>
<td align="center">

$(5-10\log_{10} 2)$ dB (theoretical, approx.)</td>
<td rowspan="3" align="center">(there is no concept of resource)</td>
<td align="center">

$(5-10\log_{10} 2)$ dB (approx.)</td>
</tr>
<tr>
<td align="center">Anti-squeezing level</td>
<td align="center">

$(10-10\log_{10} 2)$ dB (theoretical, approx.)</td>
<td align="center">

$(10-10\log_{10} 2)$ dB (approx.)</td>
</tr>
<tr>
<td align="center">Squeezing angle</td>
<td align="center">

parameter {math}`\phi` of {py:class}`~mqc3.circuit.state.HardwareConstrainedSqueezedState`</td>
</td>
<td align="center">

parameter {math}`\phi` of {py:class}`~mqc3.circuit.state.HardwareConstrainedSqueezedState`</td>
</td>
</tr>
<tr>
<td rowspan="3" align="center">Graph</td>
<td align="center">Squeezing level</td>
<td align="center">5 dB (theoretical)</td>
<td align="center">

$(5-10\log_{10} 2)$ dB (theoretical, approx.)</td>
<td align="center">5 dB</td>
<td align="center">

$(5-10\log_{10} 2)$ dB (approx.)</td>
</tr>
<tr>
<td align="center">Anti-squeezing level</td>
<td align="center">10 dB (theoretical)</td>
<td align="center">

$(10-10\log_{10} 2)$ dB (theoretical, approx.)</td>
<td align="center">10 dB</td>
<td align="center">

$(10-10\log_{10} 2)$ dB (approx.)</td>
</tr>
<tr>
<td align="center">Squeezing angle</td>
<td align="center">0</td>
<td align="center">

$\theta+\pi/2$, where $\theta$ is the parameter of {py:class}`~mqc3.graph.ops.Initialization`
</td>
<td align="center">0</td>
<td align="center">

$\theta+\pi/2$, where $\theta$ is the parameter of {py:class}`~mqc3.graph.ops.Initialization`
</td>
</tr>
<tr>
<td rowspan="3" align="center">Machinery</td>
<td align="center">Squeezing level</td>
<td align="center">5 dB (theoretical)</td>
<td></td>
<td align="center">5 dB</td>
<td></td>
</tr>
<tr>
<td align="center">Anti-squeezing level</td>
<td align="center">10 dB (theoretical)</td>
<td></td>
<td align="center">10 dB</td>
<td></td>
</tr>
<tr>
<td align="center">Squeezing angle</td>
<td align="center">0</td>
<td></td>
<td align="center">0</td>
<td></td>
</tr>
</table>
</div>

```{seealso}
See {ref}`sec:squeezing-level-and-squeezing-angle` for the definitions of squeezing level and squeezing angle.
```

At the bottom of the table, the parameters of a resource mode in the machinery are listed.
This is not a pure state, as indicated by the fact that the squeezing level and anti-squeezing level do not match.
This occurs due to optical loss and noise arising from finite squeezing.
As described in {ref}`sec:measurement-and-initialization`, the mode initialization can be performed in the machinery representation; however, this is just one of the operations that can be realized through homodyne angle configuration, so in the machinery representation, the state after initialization is not specially emphasized.

In the graph representation, the resource mode serves as a state prior to initialization.
Once initialized, the mode becomes manageable through the operations defined within the graph representation.
Parameters listed under the "Initialized" column correspond to those of the mode after initialization.
The user can control only the squeezing angle via the `theta` parameter of the {py:class}`mqc3.graph.ops.Initialization` operation.

In the circuit representation, the user considers only the modes after initialization and can specify the squeezing angle through the parameter of {py:class}`~mqc3.circuit.state.HardwareConstrainedSqueezedState`.